# 🐝 Aura Node: Interactive Worker

This notebook allows you to run an Aura Inference Node on Google Colab using Ollama and Gradio.

**Disclaimer:** For Research & Development Use Only. Do not use for commercial hosting.

In [ ]:
# @title 1. Install Dependencies
!curl -fsSL https://ollama.com/install.sh | sh
!pip install gradio requests toml

# Download frpc
import os
if not os.path.exists('frpc'):
    print("Downloading frpc...")
    !curl -L https://github.com/fatedier/frp/releases/download/v0.61.0/frp_0.61.0_linux_amd64.tar.gz -o frp.tar.gz
    !tar -xzf frp.tar.gz
    !cp frp_0.61.0_linux_amd64/frpc .
    !chmod +x frpc
    !rm -rf frp.tar.gz frp_0.61.0_linux_amd64
print("Setup complete!")

In [ ]:
# @title 2. Launch Aura Node UI
import os
import subprocess
import threading
import time
import requests
import gradio as gr
import sys
import collections
import toml
from io import StringIO

FRP_SERVER_ADDR = "aura.zae.life"
FRP_SERVER_PORT = 7000

# Logging system to redirect stdout to Gradio
class Logger:
    def __init__(self):
        # Keep the last 1000 chunks of log data
        self.logs = collections.deque(maxlen=1000)
        self.lock = threading.Lock()

    def write(self, data):
        with self.lock:
            self.logs.append(data)

    def get_logs(self):
        with self.lock:
            return "".join(self.logs)

logger = Logger()

class StreamToLogger:
    def __init__(self, stream, logger):
        self.stream = stream
        self.logger = logger

    def write(self, data):
        self.stream.write(data)
        self.logger.write(data)

    def flush(self):
        self.stream.flush()

# Prevent duplicate wrapping
if not isinstance(sys.stdout, StreamToLogger):
    sys.stdout = StreamToLogger(sys.stdout, logger)
    sys.stderr = StreamToLogger(sys.stderr, logger)

class AuraNode:
    def __init__(self):
        self.ollama_process = None
        self.frpc_process = None
        self.status = "Idle"
        self.requests_processed = 0
        self.is_running = False
        self.lock = threading.Lock()

    def start(self, model, name, token, remote_port, progress=gr.Progress()):
        with self.lock:
            if self.is_running:
                return "Already running"
            self.is_running = True
            self.status = "Processing"
        
        try:
            # 1. Start Ollama
            progress(0, desc="Starting Ollama server...")
            print("--- Starting Ollama server ---")
            if self.ollama_process is None:
                self.ollama_process = subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
                threading.Thread(target=self._read_output, args=(self.ollama_process,), daemon=True).start()

            # Wait for ollama to be up
            for i in range(30):
                try:
                    requests.get("http://localhost:11434")
                    break
                except requests.exceptions.RequestException:
                    time.sleep(1)
                    progress(i/30, desc=f"Waiting for Ollama ({i}s)")
            else:
                with self.lock:
                    self.is_running = False
                    self.status = "Error"
                return "Ollama failed to start"

            # 2. Pull model
            progress(0.1, desc=f"Pulling model {model}...")
            print(f"--- Pulling model: {model} ---")
            pull_proc = subprocess.Popen(["ollama", "pull", model], stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
            for line in pull_proc.stdout:
                print(line, end="")
            pull_proc.wait()
            
            if pull_proc.returncode != 0:
                with self.lock:
                    self.is_running = False
                    self.status = "Error"
                return f"Failed to pull {model}"

            # 3. Start frpc
            progress(0.9, desc="Connecting to Hive via frpc...")
            self._start_frpc(name, token, remote_port)
            
            with self.lock:
                self.status = "Connected"
            print("--- Aura Node is now Connected to the Hive! ---")
            return "Node Connected!"

        except Exception as e:
            print(f"Error: {e}")
            with self.lock:
                self.is_running = False
                self.status = "Error"
            return f"Error: {e}"

    def _start_frpc(self, name, token, remote_port):
        config_data = {
            "serverAddr": FRP_SERVER_ADDR,
            "serverPort": FRP_SERVER_PORT,
            "auth": {
                "method": "token",
                "token": token
            },
            "proxies": [{
                "name": name,
                "type": "tcp",
                "localIP": "127.0.0.1",
                "localPort": 11434,
                "remotePort": int(remote_port)
            }]
        }
        
        with open("frpc.toml", "w") as f:
            toml.dump(config_data, f)
        
        print(f"--- Starting frpc tunnel for {name} on remote port {remote_port} ---")
        if self.frpc_process:
            self.frpc_process.terminate()
            
        self.frpc_process = subprocess.Popen(["./frpc", "-c", "frpc.toml"], stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
        threading.Thread(target=self._read_output, args=(self.frpc_process,), daemon=True).start()

    def _read_output(self, process):
        for line in process.stdout:
            print(line, end="")
            # Increment stats on successful inference requests
            with self.lock:
                if "POST /api/generate" in line or "POST /api/chat" in line:
                    self.requests_processed += 1
                if "login to server success" in line:
                     self.status = "Connected"

    def stop(self):
        print("--- Stopping Aura Node ---")
        if self.frpc_process:
            self.frpc_process.terminate()
            self.frpc_process = None
        if self.ollama_process:
            self.ollama_process.terminate()
            self.ollama_process = None
        
        with self.lock:
            self.is_running = False
            self.status = "Idle"
        return "Node stopped"

node = AuraNode()

with gr.Blocks(title="Aura Node", theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🐝 Aura Interactive Worker Node")
    gr.Markdown("**Disclaimer:** For Research & Development Use Only. Do not use for commercial hosting.")
    
    with gr.Row():
        with gr.Column(scale=1):
            status_label = gr.Label(value=node.status, label="Brain Status")
            stats_box = gr.Number(value=node.requests_processed, label="Requests Processed")
        with gr.Column(scale=2):
            with gr.Row():
                model_input = gr.Textbox(value="mistral", label="Ollama Model")
                worker_name_input = gr.Textbox(value="aura-worker-colab", label="Worker Name")
            with gr.Row():
                remote_port_input = gr.Textbox(value="8083", label="Remote Port")
                token_input = gr.Textbox(label="Hive FRP Token", type="password")

    with gr.Row():
        start_btn = gr.Button("Start Node", variant="primary")
        stop_btn = gr.Button("Stop Node", variant="stop")

    log_output = gr.Textbox(lines=15, label="Agent Thinking / Logs", interactive=False)
    
    # Refresh logic for UI
    timer = gr.Timer(2)
    def refresh():
        with node.lock:
            status = node.status
            requests_processed = node.requests_processed
        logs = logger.get_logs()
        return status, requests_processed, logs
    timer.tick(refresh, outputs=[status_label, stats_box, log_output])

    start_btn.click(node.start, inputs=[model_input, worker_name_input, token_input, remote_port_input], outputs=[log_output])
    stop_btn.click(node.stop, outputs=[log_output])

demo.launch(share=True, inline=False)